In [ ]:
# from flask import Flask

# app = Flask(__name__)


# @app.route('/')
# def home():
#     return 'hello there'



# if __name__ == '__main__':
#     app.run()

In [ ]:
# from flask import Flask

# app = Flask(__name__)


# @app.route('/')
# def home():
#     return 'hello there'



# if __name__ == '__main__':
#     app.run(debug = True, use_reloader = True)

In [ ]:
# from flask import Flask
# import pickle
# import numpy as np

# app = Flask(__name__)

# model = 'pkl_file'

# with open(model, 'rb') as file:
#     model = pickle.load(file)

# Expected = {
# " ":{"min":3, "max":8},
# " ":{"min":68.0, "max":455.0},
# " ":{"min":46.0, "max":230.0},
# " ":{"min":1613.0, "max":5140.0},
# " ":{"min":8.0, "max":24.8},
# " ":{"min":70, "max":82},
# " ":{"min":1, "max":3}
# }

# @app.route('/api')
# def home():
#     return 'hello there, General Kenobi'



# if __name__ == '__main__':
#     app.run(debug = True, use_reloader = True)

In [ ]:
from flask import Flask, request, jsonify
import pickle
import numpy as np

app = Flask(__name__)

model = 'pkl_file'

with open(model, 'rb') as file:
    model = pickle.load(file)

Expected = {
" ":{"min":3, "max":8},
" ":{"min":68.0, "max":455.0},
" ":{"min":46.0, "max":230.0},
" ":{"min":1613.0, "max":5140.0},
" ":{"min":8.0, "max":24.8},
" ":{"min":70, "max":82},
" ":{"min":1, "max":3}
}

@app.route('/', methods = ['POST'])
def mpg_prediction():
    content = request.json
    errors = []
    for name in content:
        if name in Expected:
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            value = content[name]
            if value < expected_min or value > expected_max:
                errors.append(f'Out of bounds: {name}, has value of: {value}, but it should be between {expected_min} and {expected_max}.')
            else:
                errors.append(f'Unexpected field: {name}')
    for name in Expected:
        if name not in content:
            errors.append(f'Missing value: {name}')


    if len(errors) > 1:
        x = np.zeros((1, 7))

        x[0, 0] = content['cylinders']
        x[0, 1] = content['displacement']
        x[0, 2] = content['horsepower']
        x[0, 3] = content['weight']
        x[0, 4] = content['acceleration']
        x[0, 5] = content['year']
        x[0, 6] = content['origin']

        prediction = model.predict(x)
        mpg = round(prediction[0], 2)
        response = {'mpg': mpg}
        
    else:
        response = {'errors': errors}
    
    return jsonify(response)



if __name__ == '__main__':
    app.run(debug = True, use_reloader = True)

In [ ]:
import requests

# data = {"cylinders": 8, "displacement": 307, "horsepower": 120, "weight":3204, "acceleration": 20, "year": 76, "origin": 1}


data = {"cylinders": 7, "displacement": 370, "horsepower": 140, "weight":3204, "acceleration": 20, "year": 76, "origin": 1}

res = requests.post("http://localhost:5000", json=data)

if res.status_code == 200:
    print(f'It works: {res.text}')

else:
    print(f'It did not work: {res.text}')